In [1]:
import pandas as pd
import hashlib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import os

In [2]:
ARQUIVO = "newusuarioslogin.csv"

# Carrega os usuários do arquivo
try:
    usuarios_df = pd.read_csv(ARQUIVO)
except FileNotFoundError:
    usuarios_df = pd.DataFrame(columns=["nome", "matricula", "usuario", "senha", "curso", "faculdade", "esporte_favorito", "comida_favorita", "descricao"])


In [3]:
def hash_senha(senha):
    return hashlib.sha256(senha.encode()).hexdigest()

In [4]:
def cadastrar_usuario():
    nome = input("Nome Completo: ")
    matricula = input("Matrícula: ")

    usuario = input("Nome de usuário: ")
    global usuarios_df

    if usuario in usuarios_df['usuario'].values:
        print("Esse nome já está em uso.\n")
        return

    senha = input("Senha: ")
    curso = input("Curso: ")
    faculdade = input("Faculdade: ")
    esporte = input("Esporte favorito: ")
    comida = input("Comida favorita: ")
    descricao = input("Descreva-se: ")

    senha_hash = hash_senha(senha)

    novo_usuario = pd.DataFrame([{
        "nome": nome,
        "matricula": matricula,
        "usuario": usuario,
        "senha": senha_hash,
        "curso": curso,
        "faculdade": faculdade,
        "esporte_favorito": esporte,
        "comida_favorita": comida,
        "descricao": descricao
    }])

    #global usuarios_df
    usuarios_df = pd.concat([usuarios_df, novo_usuario], ignore_index=True)
    usuarios_df.to_csv(ARQUIVO, index=False)
    print("Usuário cadastrado com sucesso!\n")
    return usuario

In [ ]:
def recomendar_usuarios(nome_usuario, top_n=3):
    if len(usuarios_df) < 2:
        print("Poucos usuários cadastrados para recomendar.\n")
        return

    # Combina várias colunas em uma só string para cada usuário
    colunas_para_combinar = [
        "curso", "faculdade", "esporte_favorito", "comida_favorita", "descricao"
    ]
    usuarios_df["perfil_completo"] = usuarios_df[colunas_para_combinar].fillna("").agg(" ".join, axis=1)

    # Transforma os perfis combinados em vetores TF-IDF
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(usuarios_df["perfil_completo"])
    similarity = cosine_similarity(tfidf_matrix)

    # Encontra o índice do usuário-alvo
    try:
        idx = usuarios_df[usuarios_df["usuario"] == nome_usuario].index[0]
    except IndexError:
        print("Usuário não encontrado.")
        return

    # Calcula similaridade com todos, excluindo ele mesmo
    sim_scores = list(enumerate(similarity[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = [i for i in sim_scores if i[0] != idx][:top_n]

    print(f"\nUsuários recomendados para {nome_usuario}:\n")
    for i in sim_scores:
        recomendado = usuarios_df.iloc[i[0]]
        similaridade = similarity[idx][i[0]] * 100
        print(f"- {recomendado['nome']} ({similaridade:.1f}% parecido)") #
        print(f"  Curso: {recomendado['curso']}")
        print(f"  Faculdade: {recomendado['faculdade']}")
        print(f"  Esporte favorito: {recomendado['esporte_favorito']}")
        print(f"  Comida favorita: {recomendado['comida_favorita']}")
        print(f"  Descrição: {recomendado['descricao']}\n")


In [6]:
def login():
    usuario = input("Nome de usuário: ")
    senha = input("Senha: ")

    senha_hash = hash_senha(senha)

    usuer = usuarios_df[(usuarios_df['usuario'] == usuario) & (usuarios_df['senha'] == senha_hash)]

    if not usuer.empty:
        print(f"\nBem-vindo, {usuario}!\n")
        return usuario
    else:
        print("Nome de usuário ou senha incorretos.\n")
        return None

In [9]:
usuario = ''

print(f'{"=" * 30}')
print(f'{"SISTEMA UniLar":^30}')
print(f'{"=" * 30}')

while True:
    print("1 - Cadastrar novo usuário")
    print("2 - Fazer loguin")
    print("3 - Procurar residências")
    print("4 - Ver recomendações")
    print("5 - Sair")
    opcao = input("Escolha uma opção: ")

    if opcao == '1':
        usuario = cadastrar_usuario()
        print(usuario)
    elif opcao == '2':
        usuario = login()
        print(usuario)
    elif opcao == '3':
        print("\nDesculpe! Funcionalidade em desenvolvimento... :)\n")
    elif opcao == '4':
        #nome = input("Digite seu nome para ver recomendações: ")
        recomendar_usuarios(usuario)
    elif opcao == '5':
        break
    else:
        print("Opção inválida.\n")

        SISTEMA UniLar        
1 - Cadastrar novo usuário
2 - Fazer loguin
3 - Procurar residências
4 - Ver recomendações
5 - Sair

Bem-vindo, luiz!

luiz
1 - Cadastrar novo usuário
2 - Fazer loguin
3 - Procurar residências
4 - Ver recomendações
5 - Sair

Usuários recomendados para luiz:

- Rodrigo (34.0% parecido)
  Curso: Ciência da Computação
  Faculdade: UFPB
  Esporte favorito: Basquete
  Comida favorita: Lasanha
  Descrição: nan

- Carla Solane (33.4% parecido)
  Curso: Gastronomia
  Faculdade: UFPB
  Esporte favorito: Vôlei de praia
  Comida favorita: Lasanha
  Descrição: Gosto de teatro, comer sushi e correr na praia

1 - Cadastrar novo usuário
2 - Fazer loguin
3 - Procurar residências
4 - Ver recomendações
5 - Sair

Usuários recomendados para luiz:

- Rodrigo (34.0% parecido)
  Curso: Ciência da Computação
  Faculdade: UFPB
  Esporte favorito: Basquete
  Comida favorita: Lasanha
  Descrição: nan

- Carla Solane (33.4% parecido)
  Curso: Gastronomia
  Faculdade: UFPB
  Esporte 